<a href="https://colab.research.google.com/github/kdkim2000/JAkorea/blob/main/4__pytorch_______ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 import

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# gpu 사용 체크

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 설정값 지정하기

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST 데이터셋 불러오기

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 153065157.86it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 109033027.74it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49086473.30it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 22203413.48it/s]


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



# 합성곱 신경망 모델 만들기

In [5]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# 손실함수와 옵티마이저 정의하기

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 트레이닝

In [7]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/600], Loss: 0.2339
Epoch [1/5], Step [200/600], Loss: 0.0872
Epoch [1/5], Step [300/600], Loss: 0.0646
Epoch [1/5], Step [400/600], Loss: 0.0699
Epoch [1/5], Step [500/600], Loss: 0.0947
Epoch [1/5], Step [600/600], Loss: 0.2144
Epoch [2/5], Step [100/600], Loss: 0.0313
Epoch [2/5], Step [200/600], Loss: 0.0528
Epoch [2/5], Step [300/600], Loss: 0.1506
Epoch [2/5], Step [400/600], Loss: 0.0783
Epoch [2/5], Step [500/600], Loss: 0.0455
Epoch [2/5], Step [600/600], Loss: 0.0289
Epoch [3/5], Step [100/600], Loss: 0.0425
Epoch [3/5], Step [200/600], Loss: 0.0489
Epoch [3/5], Step [300/600], Loss: 0.0696
Epoch [3/5], Step [400/600], Loss: 0.0093
Epoch [3/5], Step [500/600], Loss: 0.0227
Epoch [3/5], Step [600/600], Loss: 0.0234
Epoch [4/5], Step [100/600], Loss: 0.0285
Epoch [4/5], Step [200/600], Loss: 0.0364
Epoch [4/5], Step [300/600], Loss: 0.0336
Epoch [4/5], Step [400/600], Loss: 0.0803
Epoch [4/5], Step [500/600], Loss: 0.0693
Epoch [4/5], Step [600/600], Loss:

# 테스트 진행

In [8]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Test Accuracy of the model on the 10000 test images: 98.76 %
